In [ ]:
#Instalação
pip install yfinance --user
# Install the main crewAI package
pip install crewai --user

pip install "numpy<2"

# Install the main crewAI package and the tools package
# that includes a series of helpful tools for your agents
pip install 'crewai[tools]' --user

In [18]:
import json
import os

import numpy as np
np.float_ = np.float64

from datetime import datetime

import yfinance as yf

from crewai import Agent, Task

from langchain.tools import Tool

In [19]:
def fetch_stock_price(ticket):
    stock = yf.download("AAPL", start="2023-08-08", end="2024-08-08")
    return stock

yahoo_finance_tool = Tool(
    name = "Yahoo Finance Tool",
    description = "Fetches stock prices for {ticket} from the last year about a specific stock from Yahoo Finance API",
    func= lambda ticket: fetch_stock_price(ticket)
)

In [20]:
response = yahoo_finance_tool("AAPL")

[*********************100%%**********************]  1 of 1 completed


In [21]:
print(response)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643387   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043762   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825150   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546371   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990005  201.070007  207.229996  206.990402   
2024-08-07  206.899994  213.639999  206.389999  209.820007  209.577423   

               Volume  
Date         